In [ ]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# The <a href="https://en.wikipedia.org/wiki/DPLL_algorithm">Davis-Putnam Algorithm</a>

This notebook implements the algorithm of Davis and Putnam.  Further details about this algorithm are provided in the lecture notes.

The function $\texttt{complement}(l)$ computes the complement of a literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{complement}(p) = \neg p$,
    </li>
    <li>$\texttt{complement}(\neg p) = p$.
    </li>
</ol>
As we are working with clauses that result form transforming given formulas into <em style="color:blue">conjunctive normal form</em> and these clauses do not contain $\top$ or $\bot$, we don't have to bother with $\top$ or $\bot$ in this function.

In [ ]:
def complement(l):
    'Compute the complement of the literal L.'
    if isinstance(l, str):
        return ('¬', l)
    else:
        return l[1]

In [ ]:
complement('p')

In [ ]:
complement(('¬', 'p'))

The function $\texttt{extractVariable}(l)$ extracts the variable from the literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{extractVariable}(p) = p$,
    </li>
    <li>$\texttt{extractVariable}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def extractVariable(l):
    'Extract the propositional variable from the literal l.'
    if isinstance(l, str):
        return l
    else:
        return l[1]

In [ ]:
extractVariable('p')

In [ ]:
extractVariable(('¬', 'p'))

The function $\texttt{arb}(S)$ returns an arbitrary element from the set $S$.

In [ ]:
def arb(S):
    'Return some member from the set S.'
    for x in S: return x

The function $\texttt{selectVariable}(\texttt{Clauses}, \texttt{Forbidden})$
select an arbitrary variable from some clause from the set <tt>Clauses</tt> that does not occur in the set <tt>Forbidden</tt>.

In [ ]:
def selectVariable(Clauses, Forbidden):
    return arb({ extractVariable(L) for C in Clauses for L in C } - Forbidden)

Given a set of clauses $\texttt{Cs}$ and a literal $l$, the procedure $\texttt{reduce}(C, l)$ performs all unit cuts and all unit subsumptions on clauses of $C$ that are possible using the unit clause $\{l\}$.  The resulting set of clauses is returned.  Mathematically, the function `reduce` satifies the following specification:
$$\texttt{reduce}(\texttt{Cs},l)  = 
 \Bigl\{\, C \backslash \bigl\{\overline{\,l\,}\bigr\} \;|\; C \in \texttt{Cs} \wedge \overline{\,l\,} \in C \,\Bigr\} 
       \,\cup\, \Bigl\{\, C \in \texttt{Cs} \mid \overline{\,l\,} \not\in C \wedge l \not\in C \Bigr\} \cup \bigl\{\{l\}\bigr\}.
$$


In [ ]:
def reduce(Clauses, l):
    lBar = complement(l)
    return   { C - { lBar } for C in Clauses if lBar in C }          \
           | { C for C in Clauses if lBar not in C and l not in C }  \
           | { frozenset({l}) }

$\texttt{Clauses}$ is a set of clauses.  The call $\texttt{saturate}(\texttt{Clauses})$ computes the set of those clauses that can be derived from $S$ via repeated applications of unit cuts or unit subsumptions.

In [ ]:
def saturate(Clauses):
    S     = Clauses.copy()
    Units = { C for C in S if len(C) == 1 } # set of unit clauses occurring in C
    Used  = set()                           # remember which unit clauses have already been used
    while len(Units) > 0:  # iterate as long as we derive new unit clauses
        unit  = Units.pop()
        Used |= { unit }
        l     = arb(unit)
        S     = reduce(S, l)
        Units = { C for C in S if len(C) == 1 } - Used        
    return S

The function $\texttt{solve}(\texttt{Clauses}, \texttt{Literals})$ takes a set of clauses and a set of Literals as input.  The function tries to compute a variable assignment that satisfies all clauses in $\texttt{Clauses}$.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If the set $\texttt{Clauses}$ is unsatisfiable, the set $\big\{\{\}\bigr\}$ is returned.

The argument $\texttt{Variables}$ is a set containing all those variables that have already been used to reduce the clauses.  Initially, this set is empty.

In [ ]:
def solve(Clauses, Variables):
    S      = saturate(Clauses)
    empty  = frozenset()
    Falsum = {empty}
    if empty in S:                  # S is inconsistent
        return Falsum               
    if all(len(C) == 1 for C in S): # S is trivial
        return S
    # case distinction on variable p
    p      = selectVariable(S, Variables)
    negP   = complement(p)
    Result = solve(S | { frozenset({p}) }, Variables | { p })
    if Result != Falsum:
        return Result
    return solve(S | { frozenset({negP}) }, Variables| { p })

$\mapsto$ The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets.  This is only used for pretty printing.

In [ ]:
def literal_to_str(C):
    'Convert a unit clause to a string.'
    l = arb(C)
    if l[0] == '¬':
        return f'{str(l[1])} ↦ False' 
    else:
        return f'{str(l)} ↦ True'
    
def toString(S):
    'Convert the set S of frozen sets to a string where frozen sets are written as sets.'
    if S == set():
        return f'{S} is unsolvable'
    result = '{ ' + ', '.join({ literal_to_str(C) for C in S }) + ' }'
    return result

In [ ]:
c1 = frozenset({ 'r', 'p', 's' })
c2 = frozenset({ 'r', 's' })
c3 = frozenset({ 'p', 'q', 's' })
c4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6 = frozenset({ 'p', ('¬', 'q'), 'r'})
c7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8 = frozenset({ ('¬', 'p'), ('¬', 's')})
c9 = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
c0 = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
S  = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 }
toString(solve(S, set()))